In [1]:
import gmaps
import pandas as pd
import numpy as np
import ipywidgets as widgets


In [2]:
API_KEY = ""
gmaps.configure(api_key=API_KEY)

In [3]:
def dist(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

In [4]:
LocToPath = {}
class PotholesVisualizer(object):

    def __init__(self, coordinates=None, weights=None):
        self._coordinates = coordinates

        for i in range(len(weights)):
            tup = (coordinates.iloc[i][0], coordinates.iloc[i][1])
            LocToPath[tup] = weights.iloc[i][0][11:]
        self._figure = gmaps.figure()
        self._drawing = gmaps.drawing_layer()
        self._drawing.on_new_feature(self._new_feature_callback)
        self._figure.add_layer(self._drawing)
        
        self._heatmap = gmaps.heatmap_layer(coordinates, weights["Level"]) 
        self._figure.add_layer(self._heatmap)
        
        self._image_box = widgets.Image(
            description='Image: ',
            format="jpg",
            width=500,
            height=600
        )
        self._container = widgets.VBox([self._figure, self._image_box])
        self._container.layout.align_items = "stretch"

    def _get_closest_location(self, location):
        ret = location
        close = float("infinity")
        for loc in zip(self._coordinates.lat, self._coordinates.lon):
#             print("-->> loc = ", loc, type(loc[0]))
            if dist(location, loc) < close:
                close = dist(location, loc)
                ret = loc
        return ret

    def _get_image(self, location):
        print("-> Pointed at = ", location)
        location = self._get_closest_location(location)
        path_to_img = "coordinate_csvs/Pothole.jpg"
        if location not in LocToPath:
            return None
        path_to_img = "mapped_images/" + LocToPath[location]
        try:
            return open(path_to_img, "rb").read()
        except:
            return None
    
    def _clear_image_box(self):
        self._image_box.value = b''

    def _show_image(self, location):
        image = self._get_image(location)
        if image is None:
            self._image_box.value = b''
        else:
            self._image_box.value = image
    
    def _new_feature_callback(self, feature):
        try:
            location = feature.location
        except AttributeError:
            return # Not a marker

        # Clear address box to signify to the user that something is happening
        self._clear_image_box()

        # Remove all markers other than the one that has just been added.
        self._drawing.features = [feature]

        # Compute the address and display it
        self._show_image(location)

    def render(self):
        return self._container, self._heatmap


In [5]:

weights = pd.read_csv("images_to_level.csv")
weights.drop(["Unnamed: 0"], axis=1, inplace=True)
weights = weights[:181]

df = pd.read_csv("coordinate_csvs/2.csv")
fig, heatmap = PotholesVisualizer(df[["lat", "lon"]], weights).render()

In [9]:
fig

-> Pointed at =  (29.917502836400736, 78.10603919392497)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.918451686003436, 78.1057333946228)
-> Pointed at =  (29.917776910549506, 78.10726228123576)
-> Pointed at =  (29.91882770706663, 78.10727301007182)
-> Pointed at =  (29.917926343888254, 78.1057646331451)
-> Pointed at =  (29.917679916920914, 78.10536230179287)
-> Pointed at =  (29.917679916920914, 78.10536230179287)
-> Pointed at =  (29.916949931570517, 78.10343647572017)
-> Pointed at =  (29.917080120432775, 78.1018915233276)


In [8]:
heatmap.max_intensity = 5
heatmap.dissipating = True
heatmap.point_radius = 8
heatmap.opacity = 0.7

lst = []

for i in range(181):
    
    s = weights.iloc[i][0]
    if len(s) == 22:
        s = s[:-6] + "0" + s[-6:]
    elif len(s) == 21:
        s = s[:-5] + "00" + s[-5:]
    lst.append(s)



weights["New"] = lst

len(weights)

weights = weights.sort_values("New")

weights.drop(["New"], axis=1, inplace=True)

weights.to_csv("natsorted_weights.csv")

for i in range(10):
    print(weights.iloc[i][0][11:], type(weights.iloc[i][0]))

def get_map(path_to_csv):
    
    df = pd.read_csv(path_to_csv)
    df.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)
    
    weights = np.random.normal(loc=3, scale=2, size=len(df))
    weights = [max(0, int(x)) for x in weights]
    weights = [min(6, int(x)) for x in weights]
    
    
    heatmap = gmaps.heatmap_layer(df, weights=weights)
    fig = gmaps.figure()
    fig.add_layer(heatmap)
    return fig, heatmap

fig, heatmap = get_map("coordinate_csvs/2.csv")